In [12]:
from pathlib import Path
import pyart
import numpy as np
from scipy.ndimage import uniform_filter1d
import copy
import pandas as pd

import xarray as xr
import cv2


In [13]:

from prepro.nexrad import prune_nexrad
from cappi.azran import make_cappi
from cappi.make_vv import simple_vv
from mcit.vil import get_vil_from_azran

#viewable plots
import matplotlib.pyplot as plt
from klaus_krause_cmap import get_zdr_cmap
from config import _EXAMPLEDATA_DIR



In [14]:
filename = Path(_EXAMPLEDATA_DIR, 'nexrad_level2', 'KUDX20150620_040849_V06.gz')
#filename = Path(_EXAMPLEDATA_DIR, 'nexrad_level2', 'KOAX20140603_213649_V06.gz')
#filename = Path(_EXAMPLEDATA_DIR, 'nexrad_level2', 'KEWX20210504_020040_V06')
#filename = Path(_EXAMPLEDATA_DIR, 'nexrad_level2', 'KSHV20230613_230228_V06')
#filename = Path(_EXAMPLEDATA_DIR, 'nexrad_level2', 'KTLX20160526_215113_V06')
#filename = Path(_EXAMPLEDATA_DIR, 'nexrad_level2', 'KTLX20200422_023145_V06')
#read in the Level 2 WSR-88D data as a pyart object
radar_vol = pyart.io.read_nexrad_archive(filename)
radar_vol.info()


In [15]:
#remove extra sweeps of data. Keep only data
#from the survelience cuts and one cut per volume 
prune_actions = ['surv', 'volume']
prune_vol = prune_nexrad(prune_actions, radar_vol)

#from hotspots.io.radarinfo import get_project_root, read_nexrad_radarinfo
#print(get_project_root())
#radar_info = read_nexrad_radarinfo()
#print(radar_info)


In [16]:
import matplotlib.pyplot as plt
display = pyart.graph.RadarDisplay(prune_vol)

display.plot_ppi('reflectivity', sweep=0)
plt.show()

In [17]:
#w_phase = pyart.correct.unwrap.dealias_unwrap_phase(prune_vol, unwrap_unit='volume')
#rune_vol.add_field("unwrapped_differential_phase", uw_phase)
prune_vol.info()


In [18]:
#
#Eventually our goal here is to create the entire nORPG PreProcessor stack, but at
#the moment we need the work done on the reflectivity and the zdr fields, which is
#easier and faster than the computation of KDP. So skipping the KDP computation
#we replicate the smoothing and attenuation corrections done on the ORPG
#
#we use the nORPG name to identify that we are targeting the ORPG, but we cannot
#stay totally upto date with it.
#
#basic plan:
#  unwrap_phi
pyart_vol = prune_vol
#
sweeps = pyart_vol.sweep_number['data']

#extract a sweep of phi and rhv

#print('working...will notify at end')
si= 100
ei = 200
ia = 528
#our output target
uw_phi = copy.deepcopy(pyart_vol.fields['differential_phase'])
wrap_loc = copy.deepcopy(pyart_vol.fields['differential_phase'])
#print(uw_phi)

# run the code off of the phi data, but the rhv data should match
phi_field = copy.deepcopy(pyart_vol.fields['differential_phase'])
cc_field = pyart_vol.fields['cross_correlation_ratio']
#fixme
#check for matching azimuth in rhv_field and adjust it to match the phi_field
#right now assume they match in azimuth
missing_value = phi_field["_FillValue"]    

for s in range(len(sweeps)):
    #which data do we use?
    start_index, end_index = prune_vol.get_start_end(s)
    print(s, 'sweep_num: ', sweeps[s], 'start: ', start_index, ' end: ', end_index)
    #our output target
   
    fold_in_deg = 360.0
    window_size = 30
    #half_width = int (np.floor(window_size * 0.5))
    min_cc_threshold = 0.85 #data below this value are not considered in the computation
    min_count_before_unfold = 15
    first_bin_allowed_to_fold = 100 #prevents early unfolding error
    
    folded_rays = []
    
    for a in range(start_index, end_index, 1):
        #print ('a: ', a)
        #we unwrap phase like the ORPG here
        phi_radial_raw = np.array(phi_field['data'][a])
        
        #if a == ia:
        #    print('phi_raw', phi_radial_raw[si:ei])
        
        phi_radial_raw = np.where(phi_radial_raw==missing_value, np.nan, phi_radial_raw)
        #threshold based on cc
        cc_radial_raw = np.array(cc_field['data'][a])
                                         
        phi_radial_thresh = np.where(cc_radial_raw>min_cc_threshold, phi_radial_raw, np.nan)
        
        #print(a, 'max: ', np.nanmax(phi_radial_thresh), ' thresh: ', fold_in_deg*0.8)
        
        #quick exit?
        if np.nanmax(phi_radial_thresh) < (fold_in_deg*0.8):
            uw_phi['data'][a] = phi_radial_raw.copy() 
            wrap_loc['data'][a] = np.zeros_like(phi_radial_raw)
            continue
        
        phi_radial_thresh[0] = phi_radial_raw[0]
        #fill missing with last valid value
        last_value = phi_radial_raw[0]
        #print('len:',len(phi_radial_thresh) )
        for i in range(len(phi_radial_thresh)):
            #print('i:', i, ' value: ',phi_radial_thresh[i], ' last:', last_value  )
            if np.isnan(phi_radial_thresh[i]):
                phi_radial_thresh[i] = last_value
            else:
                last_value = phi_radial_thresh[i]
            #print('i: ', i, ' final:', phi_radial_thresh[i])
            
        #This uses the dataFrame and forward fill ? FIXME
        #df = pd.DataFrame(phi_radial_thresh)
        #df_out = df.ffill(phi_radial_thresh)
        #phi_radial_thresh = fd_out.flatten()
       
        #
        phi_radial_single_fold = phi_radial_thresh + fold_in_deg
        
        #if a == ia:
            #print('phi_raw', phi_radial_raw[si:ei])
            #print('phi_thresh', phi_radial_thresh[si:ei])
            #print('cc_raw', cc_radial_raw[si:ei])
        
        #print('phi_thresh', phi_radial_thresh[100:150])
        #print('phi_fold', phi_radial_single_fold[100:150])
        
        #non-centered last 30 gates, mean value
        df = pd.DataFrame(phi_radial_thresh)
        field_mean = df.rolling(window=window_size, center=False, min_periods=3 ).mean().values
        phi_radial_mean = field_mean.flatten()
        
        
        #if a == ia:
            #print('radial_mean', phi_radial_mean[si:ei])
          

        #print('phi_radial_mean', phi_radial_mean[100:150])
        phi_diff_no_fold = np.abs(phi_radial_thresh-phi_radial_mean)
        phi_diff_single_fold = np.abs(phi_radial_single_fold-phi_radial_mean)
        
        #if a == ia:
            #print('no_fold', phi_diff_no_fold[si:ei])
            #print('no_fold', phi_diff_no_fold[si:ei])

        #unwrap_flag = 0
        #valid_count = 0
        min_valid_count = 10
        
        phi_final = copy.deepcopy(phi_radial_raw)
        
        fold_locations = np.where(phi_diff_single_fold < phi_diff_no_fold, 1, 0)
        fold_locations[0:first_bin_allowed_to_fold] = 0
        ind = np.where(fold_locations)[0]
        
        
        
        if len(ind) > min_valid_count:
            #print(a, 'fold_ind: ', ind)
            start_fold_index = ind[0]
            #print('start_fold_index', start_fold_index)
            fold_locations[0:start_fold_index] = 0
            fold_locations[start_fold_index:len(fold_locations)] = 1
            folded_rays.append(a)
          
        phi_final = np.where( (fold_locations > 0) & (phi_radial_raw < fold_in_deg*.7 ), \
                         phi_radial_raw + fold_in_deg, phi_radial_raw) 
        
        #Trim back to the original data
        #phi_final = np.where(phi_radial_raw != np.nan, phi_final, np.nan)
        
        uw_phi['data'][a] = phi_final.copy() 
        wrap_loc['data'][a] = fold_locations.copy()
        #if a == ia:
        #    print('fold', fold_locations[si:ei])
        #    print('phi_final', phi_final[si:ei])
       
        
        
    print('sweep:', s, ' unwrapped:', folded_rays)
    for r in folded_rays:
        print('ray: ',r, ' az: ', prune_vol.azimuth['data'][r])

In [19]:
#add the unwrapped Phi to the pyart object:
prune_vol.add_field('unwrapped_phi', uw_phi)
prune_vol.info()

In [20]:
ray_index = 530
# More versatile wrapper
fig, host = plt.subplots(figsize=(8,5), layout='constrained') # (width, height) in inches
# (see https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplots.html and
# .. https://matplotlib.org/stable/tutorials/intermediate/constrainedlayout_guide.html)
    
ax2 = host.twinx()
ax3 = host.twinx()

az = prune_vol.azimuth['data'][ray_index]

y = uw_phi['data'][ray_index]
x = np.arange(len(y))
    
host.set_xlim(0, 1800)
host.set_ylim(0, 720)
ax2.set_ylim(-1, 2)
ax3.set_ylim(0, 1.1)
    
host.set_title('Azimuth '+str(az))
host.set_xlabel("GateNumber")
host.set_ylabel("Unwrapped Phi")
ax2.set_ylabel("Wrap")
ax3.set_ylabel("CC")

color1, color2, color3 = plt.cm.viridis([0, .5, .9])

#y = np.where(cc_field['data'][ray_index]> 0.8, phi_field['data'][ray_index], np.nan)
#y = np.where(cc_field['data'][ray_index]> 0.8, uw_phi['data'][ray_index], np.nan)
y = uw_phi['data'][ray_index]
p1 = host.plot(x, y,    color=color1, label="Unwrapped Phi")
y = wrap_loc['data'][ray_index]
p2 = ax2.plot( x, y,    color=color2, label="Wrap")
#y = cc_field['data'][ray_index]
#p3 = ax3.plot( x, y, color=color3, label="CC")

host.legend(handles=p1+p2, loc='best')

# right, left, top, bottom
ax3.spines['right'].set_position(('outward', 60))


In [21]:

# Create figure and subplot manually
# fig = plt.figure()
# hojjst = fig.add_subplot(111)

# More versatile wrapper
fig, host = plt.subplots(figsize=(8,5), layout='constrained') # (width, height) in inches
# (see https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplots.html and
# .. https://matplotlib.org/stable/tutorials/intermediate/constrainedlayout_guide.html)
    
ax2 = host.twinx()
ax3 = host.twinx()

az = prune_vol.azimuth['data'][ray_index]
y = phi_field['data'][ray_index]
x = np.arange(len(y))
    
host.set_xlim(0, 1800)
host.set_ylim(0, 720)
ax2.set_ylim(0, 720)
ax3.set_ylim(0, 1.1)
    
host.set_title('Azimuth '+str(az))
host.set_xlabel("GateNumber")
host.set_ylabel("Phi")
ax2.set_ylabel("Unwrapped Phi")
ax3.set_ylabel("CC")

color1, color2, color3 = plt.cm.viridis([0, .5, .9])

#y = np.where(cc_field['data'][ray_index]> 0.8, phi_field['data'][ray_index], np.nan)
y = phi_field['data'][ray_index]
p1 = host.plot(x, y,    color=color1, label="Phi")
y = np.where(cc_field['data'][ray_index]> 0.8, uw_phi['data'][ray_index], np.nan)
p2 = ax2.plot( x, y,    color=color2, label="UnWrappedPhi")
#y = cc_field['data'][ray_index]
#p3 = ax3.plot( x, y, color=color3, label="CC")

host.legend(handles=p1+p2, loc='best')

# right, left, top, bottom
ax3.spines['right'].set_position(('outward', 60))

# no x-ticks                 
#host.xaxis.set_ticks([])

# Alternatively (more verbose):
# host.tick_params(
#     axis='x',          # changes apply to the x-axis
#     which='both',      # both major and minor ticks are affected
#     bottom=False,      # ticks along the bottom edge are off)
#     labelbottom=False) # labels along the bottom edge are off
# sometimes handy:  direction='in'    

# Move "Velocity"-axis to the left
# ax3.spines['left'].set_position(('outward', 60))
# ax3.spines['left'].set_visible(True)
# ax3.spines['right'].set_visible(False)
# ax3.yaxis.set_label_position('left')
# ax3.yaxis.set_ticks_position('left')

host.yaxis.label.set_color(p1[0].get_color())
ax2.yaxis.label.set_color(p2[0].get_color())
#ax3.yaxis.label.set_color(p3[0].get_color())

# For professional typesetting, e.g. LaTeX, use .pgf or .pdf
# For raster graphics use the dpi argument. E.g. '[...].png", dpi=300)'
plt.savefig("pyplot_multiple_y-axis.pdf", bbox_inches='tight')
# bbox_inches='tight': Try to strip excess whitespace
# https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.savefig.html

In [22]:
from klaus_krause_cmap import *

fig = plt.figure(figsize=(12, 10))

plt.rcParams.update(plt.rcParamsDefault)
#plt.style.use('dark_background')

plt.rcParams.update(
        {'font.size': 14.0,
         'axes.titlesize': 'x-large',
         'axes.linewidth': 2.0,
         'axes.labelsize': 'large'}
    )


xlim = [-300, 300]
ylim = [-300, 300]
axislabels=["X (km)", "Y (km)"]


sweep = 2
display = pyart.graph.RadarDisplay(prune_vol)


ax1 = fig.add_subplot(221)
ax1.set_aspect('equal')
ax1.set_facecolor('darkgrey')


display.plot_ppi(
        "reflectivity", sweep=sweep, ax=ax1, vmin=-32, vmax=95,
        mask_outside=True, cmap="pyart_NWSRef", title_flag=False,
        colorbar_label='Reflectivity (dBZ)', colorbar_flag=False, axislabels=axislabels )

display.set_limits(xlim, ylim, ax=ax1)
#display.plot_range_rings([50, 100, 150], ax=ax1, lw=1.0, ls='--', col='black')

display.plot_colorbar(extend='both', pad=0.01, shrink=1.0, label='Reflectivity (dBZ)')
#
#no figure labels for paper. Information is in the
#figure caption
#
#ax1.set_title(f"Reflectivity {radarname} {cappi_time:%Y-%m-%d %H:%M}")
ax1.set_xlabel('')
ax1.set_ylabel('Y (km)')


ax2 = fig.add_subplot(222)
ax2.set_aspect('equal')
CC_cmap = get_cc()

display.plot(
        "cross_correlation_ratio", sweep=sweep, ax=ax2, vmin=0.5, vmax=1.2,
        cmap='pyart_LangRainbow12', mask_outside=True, title_flag=False,
        colorbar_label='', colorbar_flag=False, axislabels=axislabels)
display.set_limits(xlim, ylim, ax=ax2)
#display.plot_range_rings([50, 100, 150], ax=ax2, lw=1.0, ls='--', col='black')
display.plot_colorbar(extend='both', pad=0.01, shrink=1.0, label='CC')

ax3 = fig.add_subplot(223)
ax3.set_aspect('equal')


display.plot(
        "differential_phase", sweep=sweep, ax=ax3, vmin=0, vmax=720,
        cmap='pyart_HomeyerRainbow', mask_outside=True, title_flag=False,
        colorbar_label='', colorbar_flag=False, axislabels=axislabels)
display.set_limits(xlim, ylim, ax=ax3)
#display.plot_range_rings([50, 100, 150], ax=ax2, lw=1.0, ls='--', col='black')
display.plot_colorbar(extend='both', pad=0.01, shrink=1.0, label='Phi_raw')

ax4 = fig.add_subplot(224)
ax4.set_aspect('equal')


display.plot(
        "unwrapped_phi", sweep=sweep, ax=ax4, vmin=0, vmax=720,
        cmap='pyart_HomeyerRainbow', mask_outside=True, title_flag=False,
        colorbar_label='', colorbar_flag=False, axislabels=axislabels)
display.set_limits(xlim, ylim, ax=ax4)
#display.plot_range_rings([50, 100, 150], ax=ax2, lw=1.0, ls='--', col='black')
display.plot_colorbar(extend='both', pad=0.01, shrink=1.0, label='Unwrapped Phi')

plt.tight_layout()
plt.show()
plt.close(fig)


